# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra coding portion. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_listening_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('music_listening_data')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

For this table, `session_id` and `item_in_session` represent a unique identifier for a given event. The first is created for a specific session for a user that won't be repeated,
and the second is created for each item in the session. These two are sufficient to make a non-repeating entry. Furthermore, each is needed to execute the query, so they need to be included in the **PRIMARY KEY**. 

In [ ]:
query = """
    CREATE TABLE IF NOT EXISTS session_song_details 
    (session_id int, item_in_session int, artist text, song text, song_length float,
    PRIMARY KEY (session_id, item_in_session))
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

insertion_query = """
    INSERT INTO session_song_details (session_id, item_in_session, artist, song, song_length)
    VALUES (%s, %s, %s, %s, %s)
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try: 
            session.execute(insertion_query, (
                    int(line[8]), # session id
                    int(line[3]),  # item in session
                    line[0], # artist 
                    line[9], # song name
                    float(line[5]) # song length
                )
            )
        except Exception as e:
            print("Error: Ran into a problem while trying to insert into session_song_details table")
            print(e)
        



#### SELECT query to verify that the data have been inserted into session_song_details table

In [ ]:
select_query = "SELECT artist, song, song_length FROM session_song_details WHERE session_id = 338 AND item_in_session = 4"

try: 
    rows = session.execute(select_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

`userid` and `sessionid` are needed for the query execution, so they need to be included in the **PRIMARY KEY**. 
These are bundled as a composite key. Additionally, the rows returned by the query need to be organized by the 
`iteminsession` column, so it is included in the **PRIMARY KEY** to this end as a clustering column.

In [ ]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS user_listening_data 
    (user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session))
"""


insertion_query = """
    INSERT INTO user_listening_data (user_id, session_id, item_in_session, artist, song, first_name, last_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

file = 'event_datafile_new.csv'

try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try: 
            session.execute(insertion_query, (
                    int(line[10]), # user id
                    int(line[8]), # session id
                    int(line[3]),  # item in session
                    line[0], # artist 
                    line[9], # song name
                    line[1], # first name
                    line[4]  # last name
                )
            )
        except Exception as e:
            print("Error: Ran into a problem while trying to insert into user_listening_data table")
            print(e)

 

#### SELECT query  to verify that the data has been inserted into user_listening_data table

In [ ]:
select_query = "SELECT artist, song, first_name, last_name FROM user_listening_data WHERE user_id = 10 AND session_id = 182"

try: 
    rows = session.execute(select_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

`song` is needed for query execution, so it needs to be part of the primary key. However, it is not enough to make the **PRIMARY KEY** unique, since a song will be repeated more than once, as many users can listen to the same song. In fact, the purpose of this query is to find out which different users listen to a specific song. So in view of this, a clustering column needs to be added. The most logical approach is to add the `userid`, because this will add an entry to the table for each individual user that listens to a song. 

In [ ]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS song_listening_analytics 
    (song text, user_id int, first_name text, last_name text,
    PRIMARY KEY (song, user_id))
"""

query = """
    INSERT INTO song_listening_analytics (song, user_id, first_name, last_name) 
    VALUES (%s, %s, %s, %s)
"""

try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try: 
            session.execute(query, (
                    line[9], # song name
                    int(line[10]), # user id
                    line[1], # first name
                    line[4]  # last name
                    
                )
            )
        except Exception as e:
            print("Error: Ran into a problem while trying to insert into song_listening_analytics table")
            print(e)

 #### SELECT query  to verify that the data has been inserted into song_listening_analytics table

In [ ]:
select_query = "SELECT first_name, last_name FROM song_listening_analytics WHERE song = 'All Hands Against His Own'"

try: 
    rows = session.execute(select_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)
                    

### Drop the tables before closing out the sessions

In [ ]:
drop_tables = ["DROP TABLE session_song_details", "DROP TABLE user_listening_data", "DROP TABLE song_listening_analytics"]

for query in drop_tables:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()